In [71]:
import pandas as pd
import numpy as np

In [72]:
train = pd.read_csv('hospital_stay/train_data.csv')
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [73]:
# Dropping columns to focus on specific data
drop_cols = ['Hospital_type_code', 'City_Code_Hospital', 'Ward_Type', 'Ward_Facility_Code', 'City_Code_Patient', 'Admission_Deposit']
train = train.drop(drop_cols, axis=1)
train.head()

,case_id,Hospital_code,Hospital_region_code,Available Extra Rooms in Hospital,Department,Bed Grade,patientid,Type of Admission,Severity of Illness,Visitors with Patient,Age,Stay
0,1,8,Z,3,radiotherapy,2.0,31397,Emergency,Extreme,2,51-60,0-10
1,2,2,Z,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50
2,3,10,X,2,anesthesia,2.0,31397,Trauma,Extreme,2,51-60,31-40
3,4,26,Y,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50
4,5,26,Y,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50


In [74]:
# Standardizing & simplifying column names
col_mapper = {'Hospital_code':'hospital_code',
              'Hospital_region_code':'region_code',
              'Available Extra Rooms in Hospital':'extra_rooms',
              'Department':'department',
              'Bed Grade':'bed_grade',
              'patientid':'patient_id',
              'Type of Admission':'admission_type',
              'Severity of Illness':'severity',
              'Visitors with Patient':'num_visitors',
              'Age':'age',
              'Stay':'stay'}
train = train.rename(col_mapper, axis=1)
train.head()

,case_id,hospital_code,region_code,extra_rooms,department,bed_grade,patient_id,admission_type,severity,num_visitors,age,stay
0,1,8,Z,3,radiotherapy,2.0,31397,Emergency,Extreme,2,51-60,0-10
1,2,2,Z,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50
2,3,10,X,2,anesthesia,2.0,31397,Trauma,Extreme,2,51-60,31-40
3,4,26,Y,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50
4,5,26,Y,2,radiotherapy,2.0,31397,Trauma,Extreme,2,51-60,41-50


In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   case_id         318438 non-null  int64  
 1   hospital_code   318438 non-null  int64  
 2   region_code     318438 non-null  object 
 3   extra_rooms     318438 non-null  int64  
 4   department      318438 non-null  object 
 5   bed_grade       318325 non-null  float64
 6   patient_id      318438 non-null  int64  
 7   admission_type  318438 non-null  object 
 8   severity        318438 non-null  object 
 9   num_visitors    318438 non-null  int64  
 10  age             318438 non-null  object 
 11  stay            318438 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 29.2+ MB


In [76]:
# Cleaning & transforming columns
for column in train.columns:
    if train[column].dtype == 'object':
        train[column] = train[column].str.upper()
        train[column] = train[column].str.strip()
train.head()

,case_id,hospital_code,region_code,extra_rooms,department,bed_grade,patient_id,admission_type,severity,num_visitors,age,stay
0,1,8,Z,3,RADIOTHERAPY,2.0,31397,EMERGENCY,EXTREME,2,51-60,0-10
1,2,2,Z,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50
2,3,10,X,2,ANESTHESIA,2.0,31397,TRAUMA,EXTREME,2,51-60,31-40
3,4,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50
4,5,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50


In [77]:
# Applying custom method to length-of-stay range so that calculations can be done on numerical values
def convert_to_numerical_length_of_stay(range_str):
    if range_str == 'MORE THAN 100 DAYS':
        return 120  # Assume a value greater than 100 days
    else:
        lower, upper = map(int, range_str.split('-'))
        return np.mean([lower, upper])

In [78]:
# Applying custom method to age range so that calculations can be done on numerical values
def convert_to_numerical_age(age_range):
    lower, upper = map(int, age_range.split('-'))
    return int((lower + upper) / 2)  # Taking the midpoint as the numerical value

In [79]:
# Adding new columns from custom methods
train['mean_stay'] = train['stay'].apply(convert_to_numerical_length_of_stay).astype('int64')
train['mean_age'] = train['age'].apply(convert_to_numerical_age).astype('int64')
train.head()

,case_id,hospital_code,region_code,extra_rooms,department,bed_grade,patient_id,admission_type,severity,num_visitors,age,stay,mean_stay,mean_age
0,1,8,Z,3,RADIOTHERAPY,2.0,31397,EMERGENCY,EXTREME,2,51-60,0-10,5,55
1,2,2,Z,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55
2,3,10,X,2,ANESTHESIA,2.0,31397,TRAUMA,EXTREME,2,51-60,31-40,35,55
3,4,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55
4,5,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55


In [80]:
# Identifying number of unique facilities
train['hospital_code'].nunique()

32

In [81]:
# Identifying hospitals per region
train['region_code'].value_counts()

X    133336
Y    122428
Z     62674
Name: region_code, dtype: int64

In [127]:
# Investigating the distribution of length of stay in hospitals
longest_mean_stay = train.groupby(['hospital_code']).agg({'mean_stay':'mean'}).sort_values('mean_stay', ascending=False).reset_index()
longest_mean_stay = longest_mean_stay.round(0)
longest_mean_stay.columns = ['Hospital Code','Average stay']
longest_mean_stay.head(10)

,Hospital Code,Average stay
0,2,41.0
1,22,39.0
2,16,39.0
3,20,38.0
4,21,37.0
5,25,37.0
6,29,36.0
7,26,36.0
8,32,36.0
9,1,35.0


In [109]:
# Identifying the distribution of length of stay in hospitals
train['mean_stay'].describe()

count    318438.000000
mean         32.300306
std          22.646758
min           5.000000
25%          15.000000
50%          25.000000
75%          35.000000
max         120.000000
Name: mean_stay, dtype: float64

In [126]:
# Analyzing how the severity of illness impacts length of stay
condition_stay = pd.pivot_table(train,
                               index='age',
                               columns='severity',
                               values='mean_stay',
                               aggfunc='mean')
order = ['EXTREME', 'MODERATE', 'MINOR']
condition_stay = condition_stay[order]
condition_stay = condition_stay.round()
condition_stay

severity,EXTREME,MODERATE,MINOR
age,,,
0-10,39.0,31.0,24.0
11-20,34.0,30.0,26.0
21-30,34.0,30.0,28.0
31-40,35.0,31.0,28.0
41-50,36.0,33.0,28.0
51-60,37.0,34.0,29.0
61-70,38.0,35.0,29.0
71-80,39.0,35.0,31.0
81-90,43.0,39.0,34.0


In [141]:
# Analyzing how the number of visitors impacts length of stay
visitors_stay = pd.pivot_table(train,
                               index='num_visitors',
                               columns='severity',
                               values='mean_stay',
                               aggfunc='mean')
visitors_stay = visitors_stay[order]
visitors_stay = visitors_stay.round()
visitors_stay.head()

severity,EXTREME,MODERATE,MINOR
num_visitors,,,
0,22.0,27.0,22.0
1,21.0,21.0,21.0
2,27.0,25.0,22.0
3,30.0,27.0,23.0
4,40.0,36.0,32.0


In [171]:
train.head()

,case_id,hospital_code,region_code,extra_rooms,department,bed_grade,patient_id,admission_type,severity,num_visitors,age,stay,mean_stay,mean_age
0,1,8,Z,3,RADIOTHERAPY,2.0,31397,EMERGENCY,EXTREME,2,51-60,0-10,5,55
1,2,2,Z,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55
2,3,10,X,2,ANESTHESIA,2.0,31397,TRAUMA,EXTREME,2,51-60,31-40,35,55
3,4,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55
4,5,26,Y,2,RADIOTHERAPY,2.0,31397,TRAUMA,EXTREME,2,51-60,41-50,45,55


In [172]:
# Identifying if extra hospital rooms impacts length of stay
train['extra_rooms'].describe()

count    318438.000000
mean          3.197627
std           1.168171
min           0.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          24.000000
Name: extra_rooms, dtype: float64

In [173]:
# Identifying mean length of stay in hospitals with greater than average available beds
extra_rooms = train[train['extra_rooms']>=3].sort_values('mean_stay').agg({'mean_stay':'mean'})
extra_rooms.head()

mean_stay    30.612245
dtype: float64

In [174]:
# Identifying mean length of stay in hospitals with less than average available beds
no_extra_rooms = train[train['extra_rooms']<3].sort_values('mean_stay').agg({'mean_stay':'mean'})
no_extra_rooms.head()

mean_stay    35.785503
dtype: float64

In [178]:
# Analyzing if bed grade impacts the length of stay
bed_grade = pd.pivot_table(train,
                          index='bed_grade',
                          columns='severity',
                          values='mean_stay',
                          aggfunc='mean')
bed_grade = bed_grade[order]
bed_grade = bed_grade.round()
bed_grade

severity,EXTREME,MODERATE,MINOR
bed_grade,,,
1.0,38.0,28.0,23.0
2.0,35.0,32.0,25.0
3.0,38.0,34.0,29.0
4.0,38.0,35.0,30.0


In [186]:
# Displaying length of stay based on department & severity
department_stay = pd.pivot_table(train,
                                index='department',
                                columns='severity',
                                values='mean_stay',
                                aggfunc='mean')
department_stay = department_stay.round()
department_stay = department_stay[order]
department_stay

severity,EXTREME,MODERATE,MINOR
department,,,
ANESTHESIA,34.0,31.0,27.0
GYNECOLOGY,37.0,33.0,29.0
RADIOTHERAPY,39.0,34.0,28.0
SURGERY,44.0,35.0,28.0
TB & CHEST DISEASE,35.0,31.0,27.0


In [191]:
# Investigating if age has an impact on length of stay
age_length = train.groupby('age').agg({'mean_stay':'mean'}).reset_index()
age_length.columns = ['Age', 'Lengh of Stay']
age_length = age_length.round()
age_length

,Age,Lengh of Stay
0,0-10,29.0
1,11-20,28.0
2,21-30,30.0
3,31-40,31.0
4,41-50,33.0
5,51-60,33.0
6,61-70,34.0
7,71-80,35.0
8,81-90,39.0
9,91-100,39.0
